In [233]:
import pandas as pd
import gspread
import os
from datetime import datetime
import requests

In [234]:
# Baixa e lê a planilha de projetos em curso
r = requests.get("https://geoex.com.br/Sistema/Job/Download?data=eyJKb2JJZCI6IjI3NGY2ZGRhLWEzZGItNGNkMi05NmQ3LWJjNGE5NjlhYTFlNyJ9", headers={'cookie':'_ga=GA1.1.1408546827.1679344756; TemaEscuro=true; Home.Buscar.Texto=; ASP.NET_SessionId=r4jjibpbd1byxmnxyobgib30; ConsultarNota.Numero=9101900448; ConsultarProjeto.Numero=B-0808522; .ASPXAUTH=9160AE158E39CD37666221CCC1EE172AA3D6C30D63C299E69025FAA248B8FEEE872910F8217D345BEBAFA8AD129E9CE0501D9F4092229C8EB83E3AEDE5CF07C79105BB052DAA6F88D2B248C2B15A8AC2A2F797281B8BEA7D58C52744F8AE0996B6B6CC4A02D60C9D4E387A97521E74B64CBD9D7473E2844E175A55653C8307E6F35C2B584C4EAEF6D3F1A3A273D6779B1EFDCF6B9AD5C490580F616E06896FE4B2BDC327FC24234A36003E18BA1063F8E8CD2199A2FEF65164280952C81AEE5B3634C2735A896965D31272EC9869B5D9; _ga_ZBQMHFHTL8=GS1.1.1681316552.74.0.1681316552.0.0.0'})
open('projetos em curso.zip', 'wb').write(r.content)
projetos_em_curso = pd.read_csv('projetos em curso.zip', delimiter=';', encoding='ISO-8859-1', low_memory=False)
projetos_em_curso['Projeto'] = projetos_em_curso['ProjetoText'].str.extract(r'(\d\d\d\d\d\d\d)')
projetos_em_curso['Projeto'] = projetos_em_curso['Projeto'].fillna(0)
projetos_em_curso['Projeto'] = projetos_em_curso['Projeto'].astype(int)

In [235]:
detalhamento_materiais = pd.read_excel("Detalhamento dos materiais.xlsx")
detalhamento_materiais = detalhamento_materiais.rename(columns={'codigo': 'Material'})

In [236]:
try:
    os.rename('mb52.xls', 'mb52.csv')
except FileExistsError:
    os.remove('mb52.csv')
    os.rename('mb52.xls', 'mb52.csv')
except FileNotFoundError:
    pass

mb52 = pd.read_csv('mb52.csv', sep='\t', encoding='ISO-8859-1')
mb52 = mb52.loc[~mb52['Material'].isna()]
mb52 = mb52.loc[~mb52['Dep.'].isna()]
mb52 = mb52.loc[~mb52['Cen.'].isna()]
mb52 = mb52.loc[mb52['Material'] != 'Material']
mb52 = mb52.loc[mb52['Material'] != '*']
mb52['Material'] = mb52['Material'].astype(int)
mb52['  Utilização livre'] = mb52['  Utilização livre'].str.replace(' ','')
mb52['  Utilização livre'] = mb52['  Utilização livre'].str.replace('.','')
mb52['  Utilização livre'] = mb52['  Utilização livre'].str.replace(',','.')
mb52['  Utilização livre'] = mb52['  Utilização livre'].astype(float)
mb52['Cen.'] = mb52['Cen.'].astype(int)
mb52['Dep.'] = mb52['Dep.'].astype(int)
mb52 = mb52.loc[mb52['Cen.'] == 610]

estoque = mb52[['Material', 'Cen.', 'Dep.', '  Utilização livre']]
deposito_vs_unidade = pd.DataFrame([[640, 'IBOTIRAMA'],
                        [659, 'VITÓRIA DA CONQUISTA'],
                        [660, 'JEQUIÉ'],
                        [666, 'IRECÊ'],
                        [691, 'BOM JESUS DA LAPA'],
                        [692, 'GUANAMBI'],
                        [693, 'BARREIRAS']], columns=['Dep.', 'UNIDADE'])
estoque = estoque.merge(deposito_vs_unidade, on=['Dep.'])

# estoque_vtc = estoque.loc[estoque['UNIDADE'] == 'VITÓRIA DA CONQUISTA']
# estoque_ibt = estoque.loc[estoque['UNIDADE'] == 'IBOTIRAMA']
# estoque_jeq = estoque.loc[estoque['UNIDADE'] == 'JEQUIÉ']
# estoque_irc = estoque.loc[estoque['UNIDADE'] == 'IRECÊ']
# estoque_bjl = estoque.loc[estoque['UNIDADE'] == 'BOM JESUS DA LAPA']
# estoque_gbi = estoque.loc[estoque['UNIDADE'] == 'GUANAMBI']
# estoque_brr = estoque.loc[estoque['UNIDADE'] == 'BARREIRAS']
# estoque_vtc = estoque.loc[estoque['Dep.'] == 659]
# estoque_ibt = estoque.loc[estoque['Dep.'] == 640]
# estoque_jeq = estoque.loc[estoque['Dep.'] == 660]
# estoque_irc = estoque.loc[estoque['Dep.'] == 666]
# estoque_bjl = estoque.loc[estoque['Dep.'] == 691]
# estoque_gbi = estoque.loc[estoque['Dep.'] == 692]
# estoque_brr = estoque.loc[estoque['Dep.'] == 693]

C:\Users\Hugo Viana\AppData\Local\Temp\ipykernel_8560\4241317223.py:9: DtypeWarning: Columns (1,2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  mb52 = pd.read_csv('mb52.csv', sep='\t', encoding='ISO-8859-1')


In [237]:
# Inicia serviço do Google Sheets
gs = gspread.service_account(filename='service_account.json')

In [238]:
orcamentos = gs.open_by_key('14d0b2l65tMKZll6SAlqXqRqAjIEIHwkRhJI4YVgIi88')
materiais_orcados = orcamentos.worksheet('MATERIAIS').get_all_records(value_render_option="UNFORMATTED_VALUE")
materiais_orcados = pd.DataFrame(materiais_orcados)
materiais_orcados = materiais_orcados.loc[materiais_orcados['Projeto'] != '']   
materiais_orcados = materiais_orcados.rename(columns = {'Código': 'Material', 'Quantidade': 'Qtd. Orçada'})

In [239]:
carteira = gs.open_by_key('1yZUYH8x-LzlA7OqrExA7igqin1PG8h3TWQPt0UUJqxI')
carteira = carteira.worksheet('CARTEIRA').get_all_records()
carteira = pd.DataFrame(carteira)
carteira = carteira.loc[carteira['PROJETO'] != '']
carteira = carteira.rename(columns={'PROJETO':'Projeto'})
projeto_vs_unidade = carteira.groupby(['Projeto', 'UNIDADE'], as_index=False).count()[['Projeto', 'UNIDADE']]

In [240]:
programacao = gs.open_by_key('1ztV6DYCUkhefULyxJTBiaAKLB_x5zAHgK9icFqBDvf4')
programacao = programacao.worksheet('PROGRAMAÇÃO').get_all_records()
programacao = pd.DataFrame(programacao)
programacao = programacao.loc[programacao['Projeto'] != '']
programacao['Data Execução'] = pd.to_datetime(programacao['Data Execução'], format='%d/%m/%Y')

In [241]:
try:
    os.rename('cji3.xls', 'cji3.csv')
except FileExistsError:
    os.remove('cji3.csv')
    os.rename('cji3.xls', 'cji3.csv')
except FileNotFoundError:
    pass

cji3 = pd.read_csv('cji3.csv', sep='\t', encoding='ISO-8859-1', skiprows=1)
cji3 = cji3.loc[cji3['Unnamed: 1'] != '*']
cji3['Material'] = cji3['Material'].fillna(0)
cji3['Material'] = cji3['Material'].astype(int)
cji3 = cji3.loc[cji3['Material'] != 0]
cji3 = cji3.loc[cji3['Usuário'].str.contains('STC|SIR')]

cji3['  Qtd.entr.'] = cji3['  Qtd.entr.'].str.replace('.','')
cji3['  Qtd.entr.'] = cji3['  Qtd.entr.'].str.replace(',','.')
cji3['  Qtd.entr.'] = cji3['  Qtd.entr.'].str.replace(' ','')
cji3['  Qtd.entr.'] = cji3['  Qtd.entr.'].astype(float)

cji3['Def.proj.'] = cji3['Def.proj.'].str.extract(r'(\d\d\d\d\d\d\d)').astype(int)

In [242]:
try:
    os.rename('zmm370.xls', 'zmm370.csv')
except FileExistsError:
    os.remove('zmm370.csv')
    os.rename('zmm370.xls', 'zmm370.csv')
except FileNotFoundError:
    pass

zmm370 = pd.read_csv('zmm370.csv', sep='\t', encoding='ISO-8859-1', skiprows=1)
zmm370 = zmm370.loc[zmm370['Registro Final'] != 'X']
zmm370 = zmm370.loc[(zmm370['Tipo movimento'] == 221) | (zmm370['Tipo movimento'] == 921)]
zmm370 = zmm370.loc[zmm370['Recebedor'].str.contains('3ST')]

zmm370['Qtd Necessária'] = zmm370['Qtd Necessária'].str.replace('.','')
zmm370['Qtd Necessária'] = zmm370['Qtd Necessária'].str.replace(',','.')
zmm370['Qtd Necessária'] = zmm370['Qtd Necessária'].str.replace(' ','')
zmm370['Qtd Necessária'] = zmm370['Qtd Necessária'].astype(float)

zmm370['Qtd Pendente'] = zmm370['Qtd Pendente'].str.replace('.','')
zmm370['Qtd Pendente'] = zmm370['Qtd Pendente'].str.replace(',','.')
zmm370['Qtd Pendente'] = zmm370['Qtd Pendente'].str.replace(' ','')
zmm370['Qtd Pendente'] = zmm370['Qtd Pendente'].astype(float)

zmm370['Projeto'] = zmm370['Projeto'].str.extract(r'(\d\d\d\d\d\d\d)').astype(int)
zmm370 = zmm370.rename(columns={'Qtd Pendente': 'Qtd. Reservada'})

C:\Users\Hugo Viana\AppData\Local\Temp\ipykernel_8560\1020458792.py:9: DtypeWarning: Columns (0,4,6,7,11,12,16,18,20,23) have mixed types. Specify dtype option on import or set low_memory=False.
  zmm370 = pd.read_csv('zmm370.csv', sep='\t', encoding='ISO-8859-1', skiprows=1)


In [243]:
reservas_criadas = zmm370[['Projeto', 'Material', 'Qtd. Reservada']].groupby(['Projeto', 'Material'], as_index=False).sum()
materiais_movimentados = cji3[['Def.proj.','Material', '  Qtd.entr.']].groupby(['Def.proj.', 'Material'], as_index=False).sum()
materiais_movimentados = materiais_movimentados.rename(columns={'Def.proj.': 'Projeto', '  Qtd.entr.': 'Qtd. Movimentada'})

In [244]:
data_inicio = datetime.strptime('2023-04-17', '%Y-%m-%d')
data_fim = datetime.strptime('2023-04-22', '%Y-%m-%d')

programacao_semana = programacao.loc[(programacao['Data Execução']>=data_inicio) & (programacao['Data Execução']<=data_fim)]
obras_programadas = programacao_semana[['Projeto']].groupby('Projeto').sum()

check_materiais = obras_programadas.merge(materiais_orcados, on='Projeto', how='left').groupby(['Projeto', 'Material'], as_index=False).sum()
check_materiais = check_materiais.merge(materiais_movimentados, on=['Projeto', 'Material'], how='left')
check_materiais = check_materiais.merge(reservas_criadas, on=['Projeto', 'Material'], how='left')
check_materiais = check_materiais.fillna(0)
check_materiais['Material pendente de saque'] = check_materiais['Qtd. Orçada'] - check_materiais['Qtd. Movimentada']
check_materiais = check_materiais.loc[check_materiais['Material pendente de saque'] > 0]
check_materiais['Reserva pendente'] = check_materiais['Material pendente de saque'] - check_materiais['Qtd. Reservada']
check_materiais = check_materiais.loc[check_materiais['Reserva pendente'] > 0]
check_materiais = check_materiais.merge(projeto_vs_unidade[['Projeto', 'UNIDADE']], on='Projeto')
check_materiais = check_materiais.merge(projetos_em_curso[['Projeto', 'StatusUsuarioENPUData']], on='Projeto', how='left')
check_materiais = check_materiais.merge(estoque, on=('Material', 'UNIDADE'))
check_materiais['Atendimento estoque'] = check_materiais['  Utilização livre'] - check_materiais['Material pendente de saque']
check_materiais = check_materiais.merge(detalhamento_materiais[['Material', 'TIPO DE MOVIMENTO']], on=('Material'))

In [249]:
check_materiais.to_excel('Programação '+ datetime.strftime(data_inicio, '%d-%m-%Y') + ' a ' + datetime.strftime(data_fim, '%d-%m-%Y') +'.xlsx', index=False)